In [563]:
import numpy as np
import pandas as pd
import scipy.io
from scipy.stats import itemfreq
from statistics import mode
import time

### Decision Tree class

In [564]:
class decision_tree:
    
    class node:
        def __init__(self, left, right, split_rule, is_leaf, label):
            self.left = left
            self.right = right
            self.split_rule = split_rule
            self.is_leaf = is_leaf
            self.label = label
    
    def __init__(self, max_depth=1e10):
        self.max_depth = max_depth

    def max_count(self, lst):
        return max(set(lst), key=list(lst).count)
        
    # utility function for entropy calculation
    def entropy(self, indices):
        p = itemfreq(a)[:, 1].astype(float) / len(indices)
        return -p.dot(np.log2(p))
    
    # calculate entropy the number of instances in each class in known
    def entropy_n(self, all_n):
        p = all_n / (np.sum(all_n)+1e-20)
        return -p.dot(np.log2(p+1e-20))
    
    # calculate the impurity("badness") of the specified split on the input data
    def impurity(self, left_label_hist, right_label_hist):
        Sl = np.sum(left_label_hist)
        Sr = np.sum(right_label_hist)
        return (Sl*self.entropy_n(left_label_hist) + Sr * self.entropy_n(right_label_hist)) / (Sl+Sr)
    
    # find the threshold that best split the data points with a certain feature
    # Note: <= th goes to S_left and > th goes to S_right
    def find_threshold(self, feature, labels):
        all_f = sorted(set(feature)) # sorted in ascending order
        all_l = set(labels) # list unique labels
        
        freq_mat = np.zeros([len(all_f), len(all_l)])
        for i, f in enumerate(all_f):
            for j, l in enumerate(all_l):
                freq_mat[i, j] = len(labels[np.where(labels[np.where(feature==f)]==l)])
        
        # calculate the average of two neighboring values as threshold
        # iterates from min to max
        all_threshold = (np.hstack((all_f[1:], all_f[-1])) + all_f) / 2.
        
        # in the beginning, all goes to the right node
        n_left = np.zeros([len(all_l)])
        n_right = np.sum(freq_mat, axis=0)
        n_left_sum = 0
        min_threshold = all_threshold[0]
        min_H = self.impurity(n_left, n_right)
        # loop through all threshold to find the one with the minimum impurity
        for i, th in enumerate(all_threshold):
            n_left += freq_mat[i, :]
            n_right -= freq_mat[i, :]
            H = self.impurity(n_left, n_right)
            if H < min_H:
                min_H = H
                min_threshold = th
        return min_threshold, min_H
    
    
    # find the best feature and threshold to split data points
    def segmenter(self, data, labels, m=-1):
        d = data.shape[1]
        if m == -1:
            all_features = np.arange(d)
        else:
            all_features = np.random.choice(range(d), m, replace=False)
        min_H = 1e20
        min_th = 0
        min_i = 0
        for i in all_features:
            threshold, H = self.find_threshold(data[:, i], labels)
            if H < min_H:
                min_H = H
                min_th = threshold
                min_i = i
        return min_i, min_th
    
    # the recurrence function that builds the decision tree
    def grow_tree(self, S, depth, m=-1):
        if len(set(self.labels[S])) == 1 or depth >= self.max_depth: # pure node or reach maximum depth
            return self.node(left=None, right=None, split_rule=None, is_leaf=1, label=self.max_count(self.labels[S]))
        else:
            min_i, min_th = self.segmenter(self.data[S, :], self.labels[S], m=m)
            # the following comprehension might be slow
#             Sl = [j for j, x in enumerate(self.data[:, min_i]) if x <= min_th and j in S]
#             Sr = [j for j, x in enumerate(self.data[:, min_i]) if x > min_th and j in S]
            # update: faster:
            Sl = [j for j in S if self.data[j, min_i] <= min_th]
            Sr = [j for j in S if self.data[j, min_i] > min_th]
            # another: faster:
#             Sl = np.intersect1d(np.where(self.data[:, min_i] <= min_th), S)
#             Sr = np.intersect1d(np.where(self.data[:, min_i] > min_th), S)
            if len(Sl) == 0 or len(Sr) == 0:
                return self.node(left=None, right=None, split_rule=None, is_leaf=1, \
                                 label=self.max_count(self.labels[S]))
            else:
                return self.node(left=self.grow_tree(Sl,depth+1), right=self.grow_tree(Sr,depth+1), split_rule = (min_i, min_th), \
                            is_leaf=0, label=None)
        
    # train the decision tree
    def train(self, data, labels, m=-1):
        self.data = data
        self.labels = labels
        S = np.array(range(len(labels)))
        self.root = self.grow_tree(S, 1, m=m)
        
    # predict labels of test data
    def predict(self, data, verbose=False):
        if data.ndim == 1: # special case of only 1 row (it becomes a 1d vector in numpy)
            data = np.reshape(data, [1, len(data)])
            N = 1
        else:
            N = data.shape[0]
        labels = np.zeros(N)    
            
        # predict each data point    
        for i in range(N):
            d = data[i, :]
            current_node = self.root
            # going down along the tree
            while not current_node.is_leaf: # not reach leaf yet
                idx = current_node.split_rule[0]
                th = current_node.split_rule[1]
                if d[idx] <= th:
                    current_node = current_node.left
                    if verbose:
                        print('Going left')
                else:
                    current_node = current_node.right
                    if verbose:
                        print('Going right')
            if verbose:
                print()
                
            labels[i] = current_node.label
        return labels

    # calculate the prediction accuracy
    def accuracy(self, data, true_labels):
        labels = self.predict(data, verbose=False)
        N = len(labels)
        return np.sum(labels == true_labels) / float(N)
        

### Random Forest Class

In [599]:
class random_forest:
    def __init__(self, n_trees=20, n_sample=1000, n_feature=-1, max_depth=1e10):
        self.n_trees = n_trees
        self.n_sample = n_sample
        self.n_feature = -1
        self.max_depth = max_depth
        self.trees = np.array([decision_tree(max_depth)] * n_trees)
        
    def train(self, data, labels):
        if self.n_feature == -1:
            d = data.shape[1]
            self.n_feature = int(np.sqrt(d)) # num of random features = sqrt(d) is a good guess
        for dt in self.trees:
            idx = np.random.choice(range(len(data)), self.n_sample)
            dt.train(data[idx, :], labels[idx], m=self.n_feature) # activate the random feature mode
    
    def predict(self, data, verbose=False):
        if data.ndim == 1: # special case of only 1 row (it becomes a 1d vector in numpy)
            data = np.reshape(data, [1, len(data)])
            N = 1
        else:
            N = data.shape[0]
        labels = np.zeros(N)    
            
        # predict each data point    
        for i in range(N):
            votes = np.zeros(self.n_trees)
            for j, t in enumerate(self.trees):
                votes[j] = t.predict(data[i, :], verbose=verbose)
            labels[i] = t.max_count(votes)
        return labels
    
    def accuracy(self, data, true_labels):
        labels = self.predict(data, verbose=False)
        N = len(labels)
        return np.sum(labels == true_labels) / float(N)
     

### 1. spam

In [652]:
data = scipy.io.loadmat('./spam_dist/spam_data.mat')
train_X = data['training_data']
train_y = data['training_labels'].ravel()
test_X = data['test_data']
len(train_X)

23702

#### Decision Tree

In [653]:
S = np.random.choice(range(len(train_y)), int(len(train_y)/5*4), replace=False)
train_data = train_X[S, :]
train_labels = train_y[S]
S = np.setdiff1d(range(len(train_y)), S)
validation_data = train_X[S, :]
validation_labels = train_y[S]
print(train_data.shape)

(18961, 32)


In [654]:
dt = decision_tree(15)
start = time.time()
dt.train(train_data, train_labels)
end = time.time()
print('Total training time: ', end-start)

l = dt.accuracy(validation_data, validation_labels)
print('Accuracy: ', l)

Total training time:  5.8212339878082275
Accuracy:  0.793503480278


#### Random Forest

In [657]:
n_trees=80
n_sample=1000
n_feature=15
max_depth=10
rf = random_forest(n_trees=n_trees, n_sample=n_sample, n_feature=n_feature, max_depth=n_feature)
start = time.time()
rf.train(train_data, train_labels)
end = time.time()
print('Total training time: ', end-start)

l = rf.accuracy(validation_data, validation_labels)
print('Accuracy: ', l)

Total training time:  14.86953616142273
Accuracy:  0.740139211137


#### Kaggle

In [647]:
from sklearn.feature_extraction.text import CountVectorizer
import glob
SPAM_DIR = 'spam/'
HAM_DIR = 'ham/'
TEST_DIR = 'test/'
NUM_TEST_EXAMPLES = 10000
spam_filenames = glob.glob('spam_dist/' + SPAM_DIR + '*.txt')
ham_filenames = glob.glob('spam_dist/' + HAM_DIR + '*.txt')
test_filenames = ['spam_dist/' + TEST_DIR + str(x) + '.txt' for x in range(NUM_TEST_EXAMPLES)]

all_text = []
for file in spam_filenames+ham_filenames: # use only training set data to build BOG
    with open(file, "r", encoding='utf-8', errors='ignore') as f:
        all_text.append(f.read())
        
all_test_text = []
for file in test_filenames: # use only training set data to build BOG
    with open(file, "r", encoding='utf-8', errors='ignore') as f:
        all_test_text.append(f.read())
        
vectorizer = CountVectorizer(min_df=4) # min word length=4
train_X = vectorizer.fit_transform(all_text).toarray()
test_X = vectorizer.transform(all_test_text).toarray()
train_y = np.concatenate((np.ones(len(spam_filenames)), np.zeros(len(ham_filenames))))

In [649]:
print(train_X.shape)
print(test_X.shape)
print(train_y.shape)

(23702, 41921)
(10000, 41921)
(23702,)


In [610]:
test_y = rf.predict(test_X)
df = pd.DataFrame({'Category': test_y.astype(int)})
df.index.rename('Id', inplace=True)
df.to_csv('spam_dist/spam_rf_%d_%d_%d_%d.csv' % (n_trees, n_sample, n_feature, max_depth))

### 2. census

In [612]:
train_data = pd.read_csv('./census_dist/train_data.csv')
test_data = pd.read_csv('./census_dist/test_data.csv')

print(train_data.shape)
print(test_data.shape)
train_data.head()

(32724, 15)
(16118, 14)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,label
0,59,Private,307423,9th,5,Never-married,Other-service,Not-in-family,Black,Male,0,0,50,United-States,0
1,32,Private,192965,HS-grad,9,Separated,Sales,Not-in-family,White,Female,0,0,45,United-States,0
2,19,Private,125591,Some-college,10,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,0
3,51,Without-pay,124963,Assoc-acdm,12,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,0
4,57,Self-emp-inc,146103,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,15024,0,30,United-States,1


### 3. Titanic

In [431]:
train_data = pd.read_csv('./titanic_dist/titanic_training.csv')
test_data = pd.read_csv('./titanic_dist/titanic_testing_data.csv')
train_data.head()

,survived,pclass,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,0.0,3.0,male,NaN,0.0,0.0,SOTON/OQ 392086,8.0500,NaN,S
1,0.0,1.0,male,22.0,0.0,0.0,PC 17760,135.6333,NaN,C
2,0.0,2.0,male,23.0,0.0,0.0,SC/PARIS 2133,15.0458,NaN,C
3,0.0,2.0,male,42.0,0.0,0.0,211535,13.0000,NaN,S
4,0.0,3.0,male,20.0,0.0,0.0,7534,9.8458,NaN,S
